In [2]:
import xarray as xr
import rioxarray as rio
from glob import glob

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent) + "/code/")
from utils import *

# Testing noteboook `02_convert_AGB_TB`

## Load data

#### [Ma et al. (2021)](https://www.nature.com/articles/s41559-021-01485-1)

In [3]:
# load data
ma_data = rio.open_rasterio('../results/00_preprocessing/ma_et_al_processed.nc',masked=True)

# convert from percent to fraction
ma_data.data = ma_data.data/100

# replace the shrub landcover with the max of the shrub and grass data 
ma_data.sel(landcover='shrub').data = ma_data.sel(landcover=['shrub','grass']).max(dim='landcover').data

# rename the shrub landcover to nonforest to and take the forest and nonforest landcovers
ma_data['landcover'] = ['nonforest','forest','grass']
ma_data = ma_data.sel(landcover=['forest','nonforest'])

#### [Huang et al. (2021)](https://essd.copernicus.org/articles/13/4263/2021/)

In [4]:
# load data
huang_data = rio.open_rasterio('../results/00_preprocessing/huang_et_al_processed.nc',masked=True)

# calculate RMF
huang_data = huang_data.sel(veg_part='root')/huang_data.sum(dim='veg_part')

# duplicate the data to be both for forest and nonforest
huang_data = xr.concat([huang_data,huang_data],dim='landcover')
huang_data['landcover'] = ['forest','nonforest']

## Run analysis

In [5]:

files_after = glob('../results/02_convert_AGB_TB/*.nc')
files_after = [f for f in files_after if ('100' not in f) & ('200' not in f) & ('xu_biomass' not in f) & ('chen_biomass' not in f)]

In [6]:
for file in files_after:
    matching_file = '../results/01_split_forest_nonforest/' + '_'.join(file.split('/')[-1].split('_TB')[0].split('_')[:-1]) + '.nc'
    RMF_method =  file.split('/')[-1].split('_TB')[0].split('_')[-1]

    ds = xr.open_dataarray(file)
    ref = xr.open_dataarray(matching_file)
    matching_file
    RMF = ma_data if RMF_method == 'ma' else huang_data

    RMF = RMF.rio.reproject_match(ref)
    RMF = RMF.fillna(0).where(ref[:,0,:,:].notnull())
    RMF_calc = (1-1/(ds/ref))[:,0,:,:]

    print(f'{file}: {(RMF-RMF_calc).sum().values:.2e}')

    assert (RMF-RMF_calc).sum().values < 1e-4, "FAIL"

../results/02_convert_AGB_TB/liu_biomass_xu_huang_TB.nc: 2.85e-05
../results/02_convert_AGB_TB/liu_biomass_song_ma_TB.nc: -1.11e-05
../results/02_convert_AGB_TB/liu_biomass_song_huang_TB.nc: 1.48e-05
../results/02_convert_AGB_TB/liu_biomass_CCI_ma_TB.nc: -2.40e-05
../results/02_convert_AGB_TB/liu_biomass_CCI_huang_TB.nc: 2.35e-05
../results/02_convert_AGB_TB/LVOD_xu_ma_TB.nc: 1.31e-05
../results/02_convert_AGB_TB/LVOD_xu_huang_TB.nc: 1.15e-05
../results/02_convert_AGB_TB/LVOD_song_ma_TB.nc: 8.44e-06
../results/02_convert_AGB_TB/LVOD_song_huang_TB.nc: 9.96e-06
../results/02_convert_AGB_TB/LVOD_CCI_ma_TB.nc: 1.13e-05
../results/02_convert_AGB_TB/LVOD_CCI_huang_TB.nc: 1.19e-05
../results/02_convert_AGB_TB/LVODmin_song_huang_TB.nc: 1.14e-05
../results/02_convert_AGB_TB/liu_biomass_xu_ma_TB.nc: -2.47e-05
../results/02_convert_AGB_TB/LVODmax_xu_ma_TB.nc: 1.29e-05
../results/02_convert_AGB_TB/LVODmax_xu_huang_TB.nc: 1.19e-05
../results/02_convert_AGB_TB/LVODmax_song_ma_TB.nc: 9.81e-06
../resu